In [1]:
from pyomo.environ import *

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [8]:
def solverstatus(results):
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        print('feasible')
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        print('infeasible')
    else:
        print ('Solver Status:',  results.solver.status)


        

    
model = AbstractModel()
model.i = Set(doc='set of Gen units')
model.GBconnect = Set(dimen=2)
model.lines = Set(dimen=2)
model.bus = Set()
model.node = Set(initialize=model.bus)

model.b = Param(model.i, doc='b coef of cost')
model.pmin = Param(model.i, doc='Min gen')
model.pmax = Param(model.i, doc='Max gen')

model.sbase = Param(doc='Sbase')
model.branchX = Param(model.lines, doc='Max gen')
model.branchLim = Param(model.lines, doc='Max gen')
model.Pd = Param(model.bus, doc='Max gen')



def Pbounds(model,i):
    
    return (model.pmin[i]/model.sbase, model.pmax[i]/model.sbase)

def PF(model,bus,node):
    if (bus,node)  in model.lines:
            x = -model.branchLim[bus,node]/model.sbase,model.branchLim[bus,node]/model.sbase
            print(bus,node,x)
            print("f1")
            return x
    elif (node,bus)   in model.lines:
            x = -model.branchLim[node,bus]/model.sbase,model.branchLim[node,bus]/model.sbase
           
            return x
    else:
        return (0,0)
          
    
    
model.P = Var(model.i, bounds=Pbounds, domain=Reals)
model.delta = Var(model.bus, domain=Reals, doc='angle')
model.flow = Var( model.bus, model.node, bounds = PF , domain=Reals, doc='flow')



def objFunc(model):
    return sum(model.b[i]*model.P[i] for i in model.i)
model.objective_function = Objective(rule=objFunc, sense = minimize)

    
def NodesOut_init(model, bus):
    retval = []
    for (i,j)  in model.lines:
        if i == bus and j != bus:
            retval.append(j)
    for (j,i)  in model.lines:
        if i == bus and j != bus:
            retval.append(j)
    
    print(retval)  
    return retval
model.NodesOut = Set(model.bus, initialize=NodesOut_init)


def Nodeswithgen(model, bus):
    retval = []
    for (gen,node) in model.GBconnect:
        if node == bus:
            retval.append(gen)
    return retval
model.GB = Set(model.bus, initialize=Nodeswithgen)



def linecalc_rule(model,bus,node):
    
    if (bus,node)  in model.lines:
        #print("linecalc_rule output")
        print(model.flow[bus,node] == (1/model.branchX[bus,node])*(model.delta[bus] - model.delta[node]))
        return model.flow[bus,node] == (1/model.branchX[bus,node])*(model.delta[bus] - model.delta[node])
    elif (node,bus)   in model.lines:
        #print("linecalc_rule output")
        print(model.flow[bus,node] == (1/model.branchX[node,bus])*(model.delta[bus] - model.delta[node]))
        return model.flow[bus,node] == (1/model.branchX[node,bus])*(model.delta[bus] - model.delta[node])
    else:
        return Constraint.Skip

    
model.constflowcalc = Constraint(model.bus,model.node , rule=linecalc_rule)


def balance_rule(model,bus):
    for i in model.GB[bus]:
        print(model.P[i],model.GB[bus])
   # print("Balance Rule output"
    '''for node in model.NodesOut[bus]:
        print(node,bus)'''
    #print(sum(model.P[i] for i in model.GB[bus]) - model.Pd[bus]/model.sbase == sum(model.flow[bus,node] for node in model.NodesOut[bus]))
    return sum(model.P[i] for i in model.GB[bus]) - model.Pd[bus]/model.sbase == sum(model.flow[bus,node] for node in model.NodesOut[bus])

model.constbalance = Constraint(model.bus, rule=balance_rule)



opt = SolverFactory('glpk')
instance = model.create_instance("5b.dat")
results = opt.solve(instance)




b1 b2 (-4.0, 4.0)
f1
b1 b4 (-4.0, 4.0)
f1
b1 b5 (-4.0, 4.0)
f1
b2 b3 (-4.0, 4.0)
f1
b3 b4 (-4.0, 4.0)
f1
b4 b5 (-2.4, 2.4)
f1
['b2', 'b4', 'b5']
['b3', 'b1']
['b4', 'b2']
['b5', 'b1', 'b3']
['b1', 'b4']
flow[b1,b2]  ==  35.587188612099645*(delta[b1] - delta[b2])
flow[b1,b4]  ==  32.89473684210526*(delta[b1] - delta[b4])
flow[b1,b5]  ==  156.25*(delta[b1] - delta[b5])
flow[b2,b1]  ==  35.587188612099645*(delta[b2] - delta[b1])
flow[b2,b3]  ==  92.59259259259258*(delta[b2] - delta[b3])
flow[b3,b2]  ==  92.59259259259258*(delta[b3] - delta[b2])
flow[b3,b4]  ==  33.67003367003367*(delta[b3] - delta[b4])
flow[b4,b1]  ==  32.89473684210526*(delta[b4] - delta[b1])
flow[b4,b3]  ==  33.67003367003367*(delta[b4] - delta[b3])
flow[b4,b5]  ==  33.67003367003367*(delta[b4] - delta[b5])
flow[b5,b1]  ==  156.25*(delta[b5] - delta[b1])
flow[b5,b4]  ==  33.67003367003367*(delta[b5] - delta[b4])
P[g1] GB[b1]
P[g2] GB[b1]
P[g3] GB[b3]
P[g4] GB[b4]
P[g5] GB[b5]


In [4]:
solverstatus(results)

feasible


In [5]:
instance.pprint()

9 Set Declarations
    GB : Size=5, Index=bus, Ordered=Insertion
        Key : Dimen : Domain : Size : Members
         b1 :     1 :    Any :    2 : {'g1', 'g2'}
         b2 :    -- :    Any :    0 : {}
         b3 :     1 :    Any :    1 : {'g3',}
         b4 :     1 :    Any :    1 : {'g4',}
         b5 :     1 :    Any :    1 : {'g5',}
    GBconnect : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     2 :    Any :    5 : {('g1', 'b1'), ('g2', 'b1'), ('g3', 'b3'), ('g4', 'b4'), ('g5', 'b5')}
    NodesOut : Size=5, Index=bus, Ordered=Insertion
        Key : Dimen : Domain : Size : Members
         b1 :     1 :    Any :    3 : {'b2', 'b4', 'b5'}
         b2 :     1 :    Any :    2 : {'b3', 'b1'}
         b3 :     1 :    Any :    2 : {'b4', 'b2'}
         b4 :     1 :    Any :    3 : {'b5', 'b1', 'b3'}
         b5 :     1 :    Any :    2 : {'b1', 'b4'}
    bus : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Siz

In [6]:
print("---System Data---")
print("--Power Demand at each bus--")
for pd in instance.bus:
     print(pd , instance.Pd[pd])
print("--Branch Limits--")
for bl in instance.lines:
     print(bl , instance.branchLim[bl])
print("--Line Impeadance--")
for imp in instance.lines:
     print(imp , instance.branchX[imp])

---System Data---
--Power Demand at each bus--
b1 0
b2 300
b3 300
b4 400
b5 0
--Branch Limits--
('b1', 'b2') 400
('b1', 'b4') 400
('b1', 'b5') 400
('b2', 'b3') 400
('b3', 'b4') 400
('b4', 'b5') 240
--Line Impeadance--
('b1', 'b2') 0.0281
('b1', 'b4') 0.0304
('b1', 'b5') 0.0064
('b2', 'b3') 0.0108
('b3', 'b4') 0.0297
('b4', 'b5') 0.0297


In [7]:
print("---Results---")
print(value(instance.objective_function.expr)*instance.sbase)
print("--Flow--")
for flow in instance.flow:
        if type(instance.flow[flow].value) is float and instance.flow[flow].value >= 0:
            print( flow, round(instance.flow[flow].value,5)*instance.sbase)
print("--delta--")
for b in instance.bus:
        print(b , instance.delta[b].value)
print("--generation--")
for i in instance.i:
     print(i , round(instance.P[i].value*instance.sbase,5))

---Results---
17479.896925381054
--Flow--
('b1', 'b2') 249.717
('b1', 'b4') 186.788
('b3', 'b2') 50.283
('b4', 'b3') 26.788
('b5', 'b1') 226.505
('b5', 'b4') 240.0
--delta--
b1 0.0
b2 -0.0701704109770064
b3 -0.0647398216016209
b4 -0.0567836701612193
b5 0.0144963298387807
--generation--
g1 40.0
g2 170.0
g3 323.49485
g4 0.0
g5 466.50515
